<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/MobilityFactor_ProximityMetricAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data source  

* https://github.com/ActiveConclusion/COVID19_mobility

## Google Mobility Data 

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=880eaf9c51f5f9e6584fe70f230b31099067549fdef04b12af8f34fea2ce14c8
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
google_data_url = 'https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/google_reports/mobility_report_US.csv'

google_filename = wget.download(google_data_url)

In [4]:
google_mobility_data = pd.read_csv(google_filename,low_memory=False)
google_mobility_data.head(2)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
0,Total,Total,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,Total,Total,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0


Note: This dataset has only US data, hence I am filtering state 'California' 

In [5]:
CA_GoogleMobility_data = google_mobility_data[google_mobility_data['state'] == 'California']
print(CA_GoogleMobility_data.shape)
CA_GoogleMobility_data.tail(2)

(9981, 9)


,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential
37857,California,Yuba County,2020-08-08,-11.0,17.0,NaN,NaN,-15.0,2.0
37858,California,Yuba County,2020-08-09,-1.0,19.0,NaN,NaN,-18.0,1.0


### Data Information

Google captures People's purpose of visit

* Retail and Recreation
* Grocery and Pharmacy
* Parks
* Transit stations
* Workplaces
* Residential


**Google note:** These Community Mobility Reports aim to provide insights into what has changed in response to policies aimed at combating COVID-19. The reports chart movement trends over time by geography, across different categories of places such as retail and recreation, groceries and pharmacies, parks, transit stations, workplaces, and residential.

## Apple Mobility data 

In [6]:
apple_report_url = 'https://raw.githubusercontent.com/ActiveConclusion/COVID19_mobility/master/apple_reports/apple_mobility_report_US.csv'

apple_filename = wget.download(apple_report_url)

In [7]:
apple_mobility_data = pd.read_csv(apple_filename,low_memory=False)
apple_mobility_data.tail(2)

,state,county_and_city,geo_type,date,driving,transit,walking
476014,Wyoming,Uinta County,county,2020-08-11,258.68,NaN,NaN
476015,Wyoming,Uinta County,county,2020-08-12,268.83,NaN,NaN


In [8]:
CA_AppleMobility_data = apple_mobility_data[apple_mobility_data['state'] == 'California']
print(CA_AppleMobility_data.shape)
CA_AppleMobility_data.tail(2)


(15825, 7)


,state,county_and_city,geo_type,date,driving,transit,walking
43042,California,Yuba County,county,2020-08-11,60.60,NaN,NaN
43043,California,Yuba County,county,2020-08-12,60.27,NaN,NaN


In [9]:
CA_AppleMobility_data['county'] = CA_AppleMobility_data['county_and_city']
CA_AppleMobility_data.tail(2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,state,county_and_city,geo_type,date,driving,transit,walking,county
43042,California,Yuba County,county,2020-08-11,60.60,NaN,NaN,Yuba County
43043,California,Yuba County,county,2020-08-12,60.27,NaN,NaN,Yuba County


In [10]:
CA_AppleMobility_data = CA_AppleMobility_data[['state','county','date','driving','transit','walking']]

CA_AppleMobility_data.tail(2)

,state,county,date,driving,transit,walking
43042,California,Yuba County,2020-08-11,60.60,NaN,NaN
43043,California,Yuba County,2020-08-12,60.27,NaN,NaN


### Data information

Apple mobility data has the following columns
* driving
* transit
* walking

**Apple note**: The CSV file shows a relative volume of directions requests per country/region or city compared to a baseline volume on January 13th, 2020.

Day defined as midnight-to-midnight, Pacific time. Cities represent usage in greater metropolitan areas and are stably defined during this period. In many countries/regions and cities, relative volume has increased since January 13th, consistent with normal, seasonal usage of Apple Maps. Day of week effects are important to normalize as you use this data. 

## Amalgamation of Google and Apple data

In [11]:
print(CA_GoogleMobility_data.shape)
print(CA_AppleMobility_data.shape)

(9981, 9)
(15825, 6)


In [12]:
CA_mobility_data = pd.merge(CA_GoogleMobility_data,CA_AppleMobility_data,how='outer' ,on=['state','county','date'])
CA_mobility_data.shape

(16356, 12)

### Data cleaning 

* Update NaN data 
* Remove incomplete data


In [13]:
CA_mobility_data.head(5)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
0,California,Total,2020-02-15,1.0,1.0,19.0,1.0,-1.0,0.0,30.03,NaN,NaN
1,California,Total,2020-02-16,5.0,0.0,31.0,1.0,-1.0,-1.0,6.16,NaN,NaN
2,California,Total,2020-02-17,9.0,1.0,44.0,-12.0,-38.0,8.0,11.60,NaN,NaN
3,California,Total,2020-02-18,0.0,0.0,16.0,3.0,0.0,0.0,13.94,NaN,NaN
4,California,Total,2020-02-19,0.0,0.0,13.0,1.0,0.0,0.0,14.47,NaN,NaN


In [14]:
CA_mobility_data[(CA_mobility_data['date'] == '2020-08-08') & (CA_mobility_data['county'] == 'Yuba County')]

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
9979,California,Yuba County,2020-08-08,-11.0,17.0,NaN,NaN,-15.0,2.0,83.21,NaN,NaN


In [15]:
CA_mobility_data = CA_mobility_data.fillna(0)

In [16]:
CA_mobility_data.tail(2)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
16354,California,Yuba County,2020-08-11,0.0,0.0,0.0,0.0,0.0,0.0,60.60,0.0,0.0
16355,California,Yuba County,2020-08-12,0.0,0.0,0.0,0.0,0.0,0.0,60.27,0.0,0.0


In [17]:
CA_mobility_data[CA_mobility_data['walking'] > 0]

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
10057,California,Antioch,2020-01-17,0.0,0.0,0.0,0.0,0.0,0.0,16.47,-2.06,11.92
10058,California,Antioch,2020-01-18,0.0,0.0,0.0,0.0,0.0,0.0,12.40,-2.81,20.35
10064,California,Antioch,2020-01-24,0.0,0.0,0.0,0.0,0.0,0.0,18.85,9.27,16.71
10065,California,Antioch,2020-01-25,0.0,0.0,0.0,0.0,0.0,0.0,15.70,3.43,24.58
10067,California,Antioch,2020-01-27,0.0,0.0,0.0,0.0,0.0,0.0,-0.65,24.23,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...
16279,California,Victorville-Hesperia,2020-08-08,0.0,0.0,0.0,0.0,0.0,0.0,86.44,0.00,102.69
16280,California,Victorville-Hesperia,2020-08-09,0.0,0.0,0.0,0.0,0.0,0.0,86.73,0.00,65.41
16281,California,Victorville-Hesperia,2020-08-10,0.0,0.0,0.0,0.0,0.0,0.0,50.77,0.00,88.21
16282,California,Victorville-Hesperia,2020-08-11,0.0,0.0,0.0,0.0,0.0,0.0,42.25,0.00,80.64


### Storing data to CSV 

In [35]:
CA_mobility_data.to_csv("CA_mobility.csv")

## Visualizations

In [18]:
import altair as alt

### Visualizing Los Angeles county

In [19]:
LA_county = CA_mobility_data[CA_mobility_data['county'] == 'Los Angeles County']
LA_county.shape

(213, 12)

In [20]:
SC_county = CA_mobility_data[CA_mobility_data['county'] == 'Santa Clara County']
SC_county.shape

(213, 12)

In [21]:
base_la = alt.Chart(LA_county).mark_bar().encode(
    x='monthdate(date):O',
).properties(
    width=500
)

base_sc = alt.Chart(SC_county).mark_bar().encode(
    x='monthdate(date):O',
).properties(
    width=500
)

In [22]:
LA_county.head(2)

,state,county,date,retail and recreation,grocery and pharmacy,parks,transit stations,workplaces,residential,driving,transit,walking
3186,California,Los Angeles County,2020-02-15,1.0,0.0,13.0,-1.0,-1.0,0.0,27.17,0.0,0.0
3187,California,Los Angeles County,2020-02-16,4.0,-1.0,27.0,-1.0,0.0,-1.0,2.67,0.0,0.0


In [23]:
red = alt.value("#f54242")

base_la.encode(y='retail and recreation').properties(title='Retail and Recreation') | base_la.encode(y='grocery and pharmacy',color=red).properties(title='Grocery and Pharmacy') 




alt.HConcatChart(...)

In [24]:
base_la.encode(y='parks').properties(title='Parks') | base_la.encode(y='transit stations',color=red).properties(title='Transit Stations') 



alt.HConcatChart(...)

In [25]:
base_la.encode(y='workplaces').properties(title='Workplaces') | base_la.encode(y='residential',color=red).properties(title='Residential') 



alt.HConcatChart(...)

In [26]:
base_la.encode(y='driving').properties(title='Driving') | base_la.encode(y='transit',color=red).properties(title='Transit')



alt.HConcatChart(...)

In [27]:
base_la.encode(y='walking').properties(title='Walking')

alt.Chart(...)

### Visualizing Santa clara county

In [28]:
red = alt.value("#f54242")

base_sc.encode(y='retail and recreation').properties(title='Retail and Recreation') | base_sc.encode(y='grocery and pharmacy',color=red).properties(title='Grocery and Pharmacy') 




alt.HConcatChart(...)

In [29]:
base_sc.encode(y='parks').properties(title='Parks') | base_sc.encode(y='transit stations',color=red).properties(title='Transit Stations') 


alt.HConcatChart(...)

In [30]:
base_sc.encode(y='workplaces').properties(title='Workplaces') | base_sc.encode(y='residential',color=red).properties(title='Residential') 


alt.HConcatChart(...)

In [31]:
base_sc.encode(y='driving').properties(title='Driving') | base_sc.encode(y='transit',color=red).properties(title='Transit')


alt.HConcatChart(...)

In [32]:
base_sc.encode(y='walking').properties(title='Walking')

alt.Chart(...)

In [34]:
base_sc.encode(y='residential').properties(title='Residential visits of Santa clara') |base_la.encode(y='residential',color=red).properties(title='Residential visits of LA') 

alt.HConcatChart(...)

## Data Understanding


From the visualization, People have more reesidential visits compared to transit stations, workplaces.

As per CDC guidelines, any physical activity such as walking, jogging has less chances for covid-19 infections/spreading. 
From the mobility data, we have walking, transit and driving data, Of these, we will include only residential, grocery, workplaces , transit stations, pharmacy and recreation visits into consideration.

**Note**:

The processed data is committed to github repository